In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

tf.keras.backend.clear_session()

In [2]:
from src.models import places_ontop_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

Using TensorFlow backend.


In [3]:
batch_size = 128
n_classes = 6
epochs = 100
img_size = 224
n_channels = 3

In [4]:
model = places_ontop_model.PlacesOntop_Model(batch_size, n_classes, epochs, img_size, n_channels,)

W1215 16:28:11.705587 140671186286400 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [5]:
from src.data import data

In [6]:
paths = data.PATH()
dataset_path = f'{paths.PROCESSED_DATA_PATH}/'
dataset = 'vision_based_dataset'
test_dataset_path = f'{dataset_path}/{dataset}/'

In [7]:
train_generator, validation_generator, test_generator = model.get_image_data_generator(test_dataset_path, train=True, validation=True, test=True, class_mode_test='categorical')

Found 114361 images belonging to 6 classes.
Found 6305 images belonging to 6 classes.
Found 6328 images belonging to 6 classes.


In [8]:
weights = model.get_class_weights(train_generator.classes, model)
model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# model.model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# instance_model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)

In [11]:
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Model, Sequential

from src.models import base_model
from src.models.keras_VGG16_places365.vgg16_places_365 import VGG16_Places365


def get_model(input_shape):
    vgg16_places = VGG16_Places365(
        weights='places', include_top=False, input_shape=input_shape)
    
    for l in vgg16_places.layers:
        l.trainable = False
    
    x = vgg16_places.output
    x = Flatten(name='flatten')(x)

    #x = Dense(4096, activation='relu', name='fc1')(x)
    #x = Dropout(0.5, name='drop_fc1')(x)

    #x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='drop_fc2')(x)

    x = Dense(n_classes, activation='softmax', name="predictions")(x)

    model = Model(inputs=vgg16_places.input, outputs=x)
    return model

In [12]:
input_shape = (img_size, img_size, n_channels)

In [13]:
model.model = get_model(input_shape)

In [9]:
model.show_summary()

##### PlacesOntop_Model #####
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   

In [17]:
model.evaluate_from_generator(test_dataset_path, test_generator=validation_generator)

Evaluating generator with 6305 images
Scores: [3.0671513080596924, 0.1379857212305069]



[3.0671513080596924, 0.1379857212305069]

In [18]:
model.evaluate_from_generator(test_dataset_path, test_generator=test_generator)

Evaluating generator with 6328 images
Scores: [2.5919816493988037, 0.1349557489156723]



[2.5919816493988037, 0.1349557489156723]

In [16]:
weights = model.get_class_weights(train_generator.classes, model)
model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# model.model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
# instance_model.compile(optimizer='adam', loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)

In [10]:
model.fit_from_generator(path=f'{dataset_path}/{dataset}', 
                         train_generator=train_generator, validation_generator=validation_generator,
                         test_generator=test_generator,
                         evaluate_net=False, use_model_check_point=True, use_early_stop=True, weighted=True,
                         show_activations=False,)

W1215 16:28:29.501663 140671186286400 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W1215 16:28:29.502433 140671186286400 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:352: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W1215 16:28:29.507600 140671186286400 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:353: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.

W1215 16:28:29.512035 140671186286400 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tf_real_estate_images_classification/src/models/base_model.py:354: The name tf.keras.backend.get_session is deprecated. Please use tf.co

class_weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
**** Class weights ****
[2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]
*** ** *** *** *** ** ***


W1215 16:28:29.750370 140671186286400 deprecation.py:323] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1215 16:28:30.482717 140671186286400 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
894/894 [==============================] - 1038s 1s/step - loss: 2.4015 - categorical_accuracy: 0.2928 - val_loss: 2.1764 - val_categorical_accuracy: 0.3545
Epoch 2/100
894/894 [==============================] - 1060s 1s/step - loss: 2.2469 - categorical_accuracy: 0.3689 - val_loss: 2.0527 - val_categorical_accuracy: 0.4125
Epoch 3/100
894/894 [==============================] - 1051s 1s/step - loss: 2.1950 - categorical_accuracy: 0.3923 - val_loss: 2.1657 - val_categorical_accuracy: 0.4208
Epoch 4/100
894/894 [==============================] - 1051s 1s/step - loss: 2.1688 - categorical_accuracy: 0.4039 - val_loss: 2.0708 - val_categorical_accuracy: 0.4346
Epoch 5/100
894/894 [==============================] - 1047s 1s/step - loss: 2.1490 - categorical_accuracy: 0.4133 - val_loss: 2.0666 - val_categorical_accuracy: 0.4411
Epoch 6/100
894/894 [==============================] - 1049s 1s/step - loss: 2.1337 - categorical_accuracy: 0.4212 - val_loss: 2.1136 - val_categorical_acc

In [11]:
model.model_is_trained = True

In [12]:
model.save_model()

weights [2.09728947 0.87794411 1.08704042 0.8524606  0.87239869 0.87343812]


In [17]:
model.predict_from_generator()

TypeError: predict_from_generator() missing 1 required positional argument: 'path'